In [6]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [7]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [8]:
# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Add fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.3649 - loss: 1.7366 - val_accuracy: 0.5585 - val_loss: 1.2343
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5927 - loss: 1.1621 - val_accuracy: 0.6278 - val_loss: 1.0353
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6453 - loss: 1.0010 - val_accuracy: 0.6470 - val_loss: 1.0174
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6817 - loss: 0.8994 - val_accuracy: 0.6751 - val_loss: 0.9403
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7149 - loss: 0.8086 - val_accuracy: 0.6888 - val_loss: 0.8900
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7353 - loss: 0.7495 - val_accuracy: 0.6940 - val_loss: 0.8875
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7585 - loss: 0.6920 - val_accuracy: 0.7165 - val_loss: 0.8339
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7725 - loss: 0.6516 

In [11]:
# Evaluate model performance
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

313/313 - 1s - 2ms/step - accuracy: 0.7092 - loss: 0.8619
Test accuracy: 0.7092000246047974


In [12]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [17]:
import zipfile
import io
import requests
import pandas as pd

# URL of the ZIP file
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip'

# Download the ZIP file
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Extract the specific CSV file from the ZIP archive
csv_filename = 'AirQualityUCI.csv'
with zip_file.open(csv_filename) as csv_file:
    data = pd.read_csv(csv_file, sep=';', decimal=',', header=0, usecols=range(15))

# Remove columns with non-numeric values (date and time)
data = data.drop(columns=['Date', 'Time'])

# Handle missing values using forward fill
data = data.ffill()

# Convert the dataset to numeric values (ignoring any other non-numeric columns)
data = data.apply(pd.to_numeric, errors='coerce')

# Feature scaling for numeric columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create sequences for RNN
import numpy as np

def create_sequences(data, time_steps=1):
    x, y = [], []
    for i in range(len(data) - time_steps - 1):
        x.append(data[i:(i + time_steps), :])
        y.append(data[i + time_steps, 0])  # Predicting the first feature (e.g., CO level)
    return np.array(x), np.array(y)

time_steps = 10
x, y = create_sequences(scaled_data, time_steps)

# Train-test split
train_size = int(len(x) * 0.8)
x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

In [18]:
# Define the RNN model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(50))
model.add(Dense(1))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [20]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.1648 - val_loss: 0.0309
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0388 - val_loss: 0.0296
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0330 - val_loss: 0.0236
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0321 - val_loss: 0.0219
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0276 - val_loss: 0.0206
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0242 - val_loss: 0.0205
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0240 - val_loss: 0.0202
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0229 - val_loss: 0.0202
Epoch 9/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0224 - val_loss: 0.0202
Epoch 10/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0208 - val_loss: 0.0220


In [21]:
# Evaluate model performance
train_loss = model.evaluate(x_train, y_train, verbose=0)
test_loss = model.evaluate(x_test, y_test, verbose=0)
print(f"Train Loss: {train_loss}, Test Loss: {test_loss}")

Train Loss: 0.022923704236745834, Test Loss: 0.02200019732117653


In [22]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

In [23]:
# Load 20 Newsgroups dataset
newsgroups = fetch_20newsgroups(subset='all')
x = newsgroups.data
y = newsgroups.target

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=2000)
x = vectorizer.fit_transform(x).toarray()

# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [24]:
# Define the MLP model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(20, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4681 - loss: 2.0484 - val_accuracy: 0.8085 - val_loss: 0.6422
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8981 - loss: 0.3870 - val_accuracy: 0.8228 - val_loss: 0.5794
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9510 - loss: 0.2011 - val_accuracy: 0.8263 - val_loss: 0.5869
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9794 - loss: 0.1071 - val_accuracy: 0.8210 - val_loss: 0.6377
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9921 - loss: 0.0499 - val_accuracy: 0.8196 - val_loss: 0.6799
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9966 - loss: 0.0283 - val_accuracy: 0.8175 - val_loss: 0.7418
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9987 - loss: 0.0158 - val_accuracy: 0.8233 - val_loss: 0.7631
Epoch 8/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9986 - loss: 0.0107 - val_accuracy: 0

In [27]:
# Evaluate model performance
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

118/118 - 1s - 8ms/step - accuracy: 0.8119 - loss: 0.8799
Test accuracy: 0.8119363188743591
